# Summary

Starting point for a supervised learning model for Ookla speed tiles. The data comes from a combination of 
Ookla Open Data speed tests and Statistics Canada information, including 2016 census population data and census boundaries (shapefiles). 


In [1]:
import sys
sys.path.append("..")

In [2]:
import src.config

In [3]:
from src.datasets.loading import statcan, ookla

In [4]:
import numpy as np 
import pandas as pd
import geopandas as gp

In [5]:
from sklearn import preprocessing, pipeline, compose
from sklearn import linear_model, model_selection, svm
from sklearn import metrics

In [6]:
import matplotlib.pyplot as plt 

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-4yww8lr4 because the default path (/home/jovyan/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


## Load 
Load some of the available data. The census population data and StatCan boundaries are automatically loaded from 
the StatCan website. The overlays and tile geometries/speeds need to pre-computed and saved to the overlays directory and data directories. 

### Load All Unique Tile Gemoetries

In [7]:
ookla_tiles = ookla.canada_tiles()

### Load Census Population Information

In [8]:
da_pops = statcan.dissemination_areas_populations()

/home/jovyan/src/datasets/loading/statcan.py:234: DtypeWarning: Columns (3,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(POP_FILE)


### Labelling Tiles
Generate labels from geometric overlay of the Ookla tiles and Statistics Canada Dissemination Areas (DA). 
Label each tile with the information from the StatCan areas based on which DA the tile overlaps the most with.

In [9]:
o = gp.read_file(src.config.OVERLAYS_DIR / 'tile_das_overlay') #this can take a few minutes to load.
tile_da_label = o.dropna(subset=['DAUID','quadkey']).sort_values(by=['quadkey','tile_frac'],ascending=False).drop_duplicates(subset='quadkey', keep='first')
tile_da_label['quadkey'] = tile_da_label['quadkey'].astype(int)
tile_da_label['DAUID'] = tile_da_label['DAUID'].astype(int)

### Speed Test Data
Load in the previous 4 quarters of data. Since we're currently in Q3 of 2022, the most recent quarter is Q2 
so we can slice the files listed to grab those. Subsequently, we'll calculate weighted averages for individual tiles and use those as representative speeds for our model.

In [10]:
last_4_quarters = ookla.speed_data(ookla.available_files().loc[('fixed',2021,3):('fixed',2022,2)].path)

In [ ]:
down = last_4_quarters.groupby('quadkey').apply(lambda s:np.average(s.avg_d_kbps, weights=s.tests)).rename('avg_d_kbps')
up = last_4_quarters.groupby('quadkey').apply(lambda s:np.average(s.avg_u_kbps, weights=s.tests)).rename('avg_u_kbps')
tests = last_4_quarters.groupby('quadkey')['tests'].sum()
devices = last_4_quarters.groupby('quadkey')['devices'].sum()
last4_agg = pd.concat([down, up, tests, devices],axis=1)

### Merge All The Data
It's a bit messy, but we're merging several tables and removing a few of the redundant or non-useful 
columns as we go through. At the end the `features_table` variable will have all of the 
tiles within census areas labelled by what type of Census Subdivision, Dissemination Area, Population Centre, etc. they are in, as well as population information for the DA (smallest area with populations available) and the speed test averages over the last 4 quarters.

In [ ]:
## merge dissemination area (DA) populations with ookla tiles (already combined with other statcan data)
features_table = tile_da_label.merge(da_pops, on='DAUID', how='left')
features_table['DAPOP'] = features_table['DAPOP'].fillna(0).astype(int)
del features_table['GEO_NAME']
features_table = pd.DataFrame(features_table)
del features_table['geometry']
features_table['POP_DENSITY'] = features_table['DAPOP']/features_table['das_area']*1000**2 #people per square kilometer

# take all ookla tiles, merge the speeds data and tile labels and populations
features_table = ookla_tiles.merge(last4_agg, on='quadkey').merge(features_table, on='quadkey')

# compute spatial joins to identify if area is a population centre
pop_info = statcan.boundary('population_centres').to_crs('epsg:4326')
pop_info = pop_info[['PCUID', 'PCNAME', 'PCTYPE', 'PCPUID', 'PCCLASS', 'geometry']] ##removes some redundant cols from DAs
features_table = features_table.sjoin(pop_info, how='left')
del features_table['index_right']
features_table = features_table.sort_values(by=['PCUID','quadkey']).drop_duplicates(subset=['quadkey']) #keep tiles where overlap was true

### Categorize All the Columns
All the columns from our joins above can be roughly split into categories based on the type of 
data and how you might use them in a simple supervised learning problem. These are broken down as follows:

In [ ]:
pkey = 'quadkey'
geometry = 'geometry'
id_and_names = ['DAUID', 'CDUID', 'CDNAME', 'CCSUID', 'CSDNAME', 'CMAUID', 'CMAPUID', 'CMANAME', 
'CCSNAME', 'CSDUID', 'ERUID', 'ERNAME', 'CTUID', 'CTNAME', 'ADAUID', 
'PCUID', 'PCNAME', 'PCPUID', 'SACCODE',] ##SACCODE is half a category half ID values

categorical_labels = [
    #'PRUID', #PRUID is redundant with PRNAME
    'PRNAME', 'CDTYPE', 
    'CSDTYPE',  
    'SACTYPE', 
    'CMATYPE', 'PCTYPE', 'PCCLASS',
]
numerical_vars = [
    'tests', 'devices',
    'das_area', 'tile_area', 'tile_frac',  'das_frac', 
    'DAPOP','POP_DENSITY'
]
target_vars = ['avg_d_kbps', 'avg_u_kbps']

In [ ]:
col_subset = [pkey] + categorical_labels + numerical_vars + target_vars
features_table.loc[:,col_subset].set_index('quadkey')

In [ ]:
features_table.info()#features_table.to_csv("Features.csv")

In [ ]:
#As we want to wokr with the MBps speed adding a column with MBPS of up and down streams.
features_table["avg_d_mbps"]=features_table["avg_d_kbps"]//1000.0
features_table["avg_u_mbps"]=features_table["avg_u_kbps"]//1000.0
features_table.describe()
#features_table.to_csv("Features.csv")

In [ ]:
#Finding unique values for each columns
for i in range(0,38):
    print("-------Finding unique values in column ----------- "+ features_table.columns[i])
    print(features_table[features_table.columns[i]].unique())

In [ ]:
import numpy as np 
from pandas import DataFrame
import seaborn as sns
#features_table_realtion = features_table.drop(columns=["quadkey","geometry"])
features_table_realtion = features_table.filter(['avg_d_mbps','avg_u_mbps','tests','devices','POP_DENSITY'], axis=1).head(1000)
sns.heatmap(features_table_realtion, annot=True)

In [ ]:
#Mean and Standard dev for each provience along with total size of location in each provience0) 
Feature_all_downspeed = features_table.groupby("PRNAME")["avg_d_mbps"].agg(['size','mean','std']).reset_index()
Feature_all_downspeed.columns = ["Proviences","Size_Total","Mean_Download_Speed","std_Download_Speed"]
Feature_all_downspeed.head(13)

In [ ]:
Feature_all_upspeed = features_table.groupby("PRNAME")["avg_u_mbps"].agg(['size','mean','std']).reset_index()
Feature_all_upspeed.columns = ["Proviences","Size_Total","Mean_Upload_Speed","STD_Upload_Speed"]
Feature_all_upspeed.head(13)

#TODO: Size total will be used to find the gap between total location having internet vs Total location not meeting the speed criteria the GAP.

frames=[Feature_all_downspeed,Feature_all_upspeed]
result = pd.concat(frames,axis=1)
result = result.T.drop_duplicates().T
result.head(13)

In [ ]:
#Similarly repeating process for internet criteria
#Finding how many locations does not meet the criteria of min up/download speed by provience
Query_up_down_speed = features_table.query('avg_d_mbps < 50 | avg_u_mbps < 10')

#For Download speeed
Query_downspeed = Query_up_down_speed.groupby("PRNAME")["avg_d_mbps"].agg(['size','mean','std',]).reset_index()
Query_downspeed.columns = ["Proviences","Crt_Size_Total","Crt_Mean_Download_Speed","Crt_std_Download_Speed"]
Query_downspeed.head(30)

In [ ]:
#For upload speed
Query_upspeed = Query_up_down_speed.groupby("PRNAME")["avg_u_mbps"].agg(['size','mean','std']).reset_index()
Query_upspeed.columns = ["Proviences","Crt_Size_Total","Crt_Mean_Up_Speed","Crt_std_Up_Speed"]
Query_upspeed.head(30)

In [ ]:
frames_crt=[Query_downspeed,Query_upspeed]
result_crt = pd.concat(frames_crt,axis=1)
result_crt = result_crt.T.drop_duplicates().T
result_crt.head(13)

In [ ]:
#Merging boeth the result tables having all the mean,std,size details for actual and expected criteria
frames_final=[result,result_crt]
result_find_gap = pd.concat(frames_final,axis=1)
result_find_gap = result_find_gap.T.drop_duplicates().T
result_find_gap.head(13)

In [ ]:
#Adding percentage column to find the gap and better understanding
result_find_gap["Percentage_gap"] = (result_find_gap["Crt_Size_Total"]/result_find_gap["Size_Total"])*100
result_find_gap.to_csv("Gap Analysis.csv")
result_find_gap.sort_values(by="Percentage_gap", ascending=False,inplace=True)
result_find_gap.head(13)

import plotly.express as px
fig = px.bar(result_find_gap, x='Proviences',y='Percentage_gap')
fig.show()

In [10]:
for_visualization_gap = pd.read_csv("./data/Gap_Analysis.csv")
for_visualization_gap.head()

,Unnamed: 0,Proviences,Size_Total,Mean_Download_Speed,std_Download_Speed,Mean_Upload_Speed,STD_Upload_Speed,Crt_Size_Total,Crt_Mean_Download_Speed,Crt_std_Download_Speed,Crt_Mean_Up_Speed,Crt_std_Up_Speed
0,0,Alberta,46933,88.960220,99.179564,33.174589,51.976972,28645,27.335870,29.772997,6.957689,8.556512
1,1,British Columbia / Colombie-Britannique,37060,150.566838,124.436208,64.355235,72.662968,11520,30.925174,34.779705,9.908247,12.759966
2,2,Manitoba,20194,81.436318,86.908821,31.959493,59.413703,12260,30.634502,33.202787,6.107259,11.330834
3,3,New Brunswick / Nouveau-Brunswick,13229,141.915715,138.299939,39.408723,58.845470,5370,28.741527,38.468404,5.584171,11.586166
4,4,Newfoundland and Labrador / Terre-Neuve-et-Lab...,6747,141.556544,120.608244,43.766859,63.201958,2912,56.487294,81.856997,5.619849,8.362949
